In [2]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from typing import List, Dict
import math
import random


In [3]:
df = pd.read_csv('train.csv')
df

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1
...,...,...,...,...,...,...,...,...,...
2024,2024,Please edit your post so it is readable. These...,No legal advice: Do not offer or request legal...,relationships,"I'm not ok with this in anyway, and think you ...",See a lawyer under the guise that you want thi...,"This is just untrue. OP is 13, not an adult -...",Why should I care about all the bicyclists I i...,1
2025,2025,"Yes, and in a right to work state they can eve...",No legal advice: Do not offer or request legal...,legaladvice,Move as much of your assets as you can offshor...,We have great consumer protection laws. There'...,"LPT piratebay, transmission, vpn. Get the musi...","It's not so much that I killed them, it's that...",0
2026,2026,**HD** Streams: |ENG **HD**[ Watch here..PC & ...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,stitopdisca1987.tumblr.com - sex Take girl for...,this girl get sex going to to old http://mrk....,NO ADS | NO ADS | NO ADS\n\nWe show all SOCCER...,[So great! Thanks for sharing.](http://www.che...,1
2027,2027,No. Not when doing so obviously presents a saf...,No legal advice: Do not offer or request legal...,politics,SHE ISNT A BIRTHING CHAMBER BUT EQUALLY THE BA...,"Jail? What are you, ten years old? If they pro...",Who cares about that when I can keep raping in...,send me a private message; I may be able to he...,1


In [4]:
def print_df(df):
    for i in range(2):
        print(f"RULE {df['rule'].iloc[i]} SUBREDDIT {df['subreddit'].iloc[i]}")
        print(f"BODY {df['body'].iloc[i]}")
        print(f"positive 1 {df['positive_example_1'].iloc[i]} \npositive 2 {df['positive_example_2'].iloc[i]}")
        print(f"negative 1 {df['negative_example_1'].iloc[i]} \nnegative 2 {df['negative_example_2'].iloc[i]}")
        print("--"*50)
print_df(df)    


RULE No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. SUBREDDIT Futurology
BODY Banks don't want you to know this! Click here to know more!
positive 1 If you could tell your younger self something different about sex, what would that be?

i AM IN A CONTEST TO WIN FUNDING FOR MY SEX POSITIVE FILM: VOTE HERE:

http://sheknows.offerpop.com/campaign/813112/entry/v144417 
positive 2 hunt for lady for jack off in neighbourhood http://url.inmusi.com/gakq
negative 1 Watch Golden Globe Awards 2017 Live Online in HD Coverage without ADS (VIP STREAMS)
=

HD STREAM QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858)
=

HD BROADCASTING QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858)
=

Mobile Compatibility: YES
=

NO ADS | NO ADS | ADS
=
 
negative 2 DOUBLE CEE x BANDS EPPS - "BIRDS"

DOWNLOAD/STREAM:

http://music.theblacksmithed.com/download/birds/
-------------------------

In [5]:
## possible cleaning ,
"""
 caps , space , puntatiom , links --> how to handle ?
"""

'\n caps , space , puntatiom , links --> how to handle ?\n'

In [6]:
# clean spaces use regex 
import re

# Remove extra spaces from all string columns
def clean_spaces(df):
    str_cols = df.select_dtypes(include=['object']).columns
    for col in str_cols:        
        df[col] = df[col].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    return df

df = clean_spaces(df)

In [7]:
print_df(df)

RULE No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. SUBREDDIT Futurology
BODY Banks don't want you to know this! Click here to know more!
positive 1 If you could tell your younger self something different about sex, what would that be? i AM IN A CONTEST TO WIN FUNDING FOR MY SEX POSITIVE FILM: VOTE HERE: http://sheknows.offerpop.com/campaign/813112/entry/v144417 
positive 2 hunt for lady for jack off in neighbourhood http://url.inmusi.com/gakq
negative 1 Watch Golden Globe Awards 2017 Live Online in HD Coverage without ADS (VIP STREAMS) = HD STREAM QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858) = HD BROADCASTING QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858) = Mobile Compatibility: YES = NO ADS | NO ADS | ADS = 
negative 2 DOUBLE CEE x BANDS EPPS - "BIRDS" DOWNLOAD/STREAM: http://music.theblacksmithed.com/download/birds/
----------------------------------

In [14]:
# pip install sentence-transformers
from sentence_transformers import SentenceTransformer, InputExample, losses, SentencesDataset
from torch.utils.data import DataLoader

model = SentenceTransformer("bert-base-uncased")
train_examples = [
    InputExample(texts=["a red apple", "green apples are tasty", "the capital of France is Paris"]),
    # texts=[anchor, positive, negative]
]
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
train_loss = losses.TripletLoss(model=model, distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE)

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=100)
emb = model.encode(["your sentence here" , 'fsf' ,'fdfdgg'], convert_to_tensor=True, normalize_embeddings=True)


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [8]:
# emb = model.encode(["your sentence here"], convert_to_tensor=True, normalize_embeddings=True)
print(emb.shape)

torch.Size([3, 768])


In [4]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer("bert-base-uncased")

# Count trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


ImportError: cannot import name 'GenerationMixin' from 'transformers.generation' (c:\Users\manis\anaconda3\envs\pytorch\Lib\site-packages\transformers\generation\__init__.py)

In [3]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("bert-base-uncased", cache_folder="./model_cache")

ImportError: cannot import name 'GenerationMixin' from 'transformers.generation' (c:\Users\manis\anaconda3\envs\pytorch\Lib\site-packages\transformers\generation\__init__.py)

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("bert-base-uncased")
print(model[0].auto_model)


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [10]:
emb = model.encode(["[I wanna kiss you all over! Stunning!](http://www.oilflush.life/2017/01/26/6/) "], convert_to_tensor=True, normalize_embeddings=True)
emb1 = model.encode(['#Rapper 🚨Straight Outta Cross Keys SC 🚨YouTube Search Beanie 864 Click Link BELOW To Hear Hit Single "Ah Man" Beanie 864 FEAT King Kota (King Kota Is Only 15!) Lit 🌡🔥👍💵💯Fr Fr https://youtu.be/tLqbV1Jmt5Y '], convert_to_tensor=True, normalize_embeddings=True)
emb3 = model.encode([" No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. SUBREDDIT soccerstreams"], convert_to_tensor=True, normalize_embeddings=True)

In [11]:
import torch
from torch.nn.functional import cosine_similarity
for i in range(3):
    emb = model.encode([f"{df['subreddit'][i]} [SEP] {df['rule'][i]} [SEP] {df['body'][i]}"], convert_to_tensor=True, normalize_embeddings=True)
    emb1 = model.encode([f"{df['subreddit'][i]} [SEP] {df['rule'][i]} [SEP] {df['positive_example_1'][i]}"], convert_to_tensor=True, normalize_embeddings=True)
    emb2 = model.encode([f"{df['subreddit'][i]} [SEP] {df['rule'][i]} [SEP] {df['negative_example_1'][i]}"], convert_to_tensor=True, normalize_embeddings=True)
    dot_01 = torch.matmul(emb, emb1.T)
    cos_01 = cosine_similarity(emb, emb1).item()
    dist_01 = torch.norm(emb - emb1).item()
    dot_02 = torch.matmul(emb, emb2.T)
    cos_02 = cosine_similarity(emb, emb2).item()
    dist_02 = torch.norm(emb - emb2).item()
    print(df["rule_violation"][i])
    print(f"Dot(emb, emb1): {dot_01.item():.4f}, Cosine: {cos_01:.4f}, Distance: {dist_01:.4f}")
    print(f"Dot(emb, emb2): {dot_02.item():.4f}, Cosine: {cos_02:.4f}, Distance: {dist_02:.4f}")
    print("-"*10)

0
Dot(emb, emb1): 0.8720, Cosine: 0.8720, Distance: 0.5061
Dot(emb, emb2): 0.7622, Cosine: 0.7622, Distance: 0.6896
----------
0
Dot(emb, emb1): 0.9287, Cosine: 0.9287, Distance: 0.3775
Dot(emb, emb2): 0.8627, Cosine: 0.8627, Distance: 0.5241
----------
1
Dot(emb, emb1): 0.8487, Cosine: 0.8487, Distance: 0.5500
Dot(emb, emb2): 0.8669, Cosine: 0.8669, Distance: 0.5159
----------


In [8]:
print(len(df["body"]))

2029


In [9]:
df[["subreddit","rule"]].value_counts()

subreddit        rule                                                                                                   
legaladvice      No legal advice: Do not offer or request legal advice.                                                     210
soccerstreams    No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.    138
AskReddit        No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.    119
relationships    No legal advice: Do not offer or request legal advice.                                                     105
personalfinance  No legal advice: Do not offer or request legal advice.                                                     102
                                                                                                                           ... 
relationships    No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not

In [10]:
### dmodel

import torch 
import torch.nn as nn   
from sentence_transformers import SentenceTransformer
from typing import List

class TripletEncoder(nn.Module):
    def __init__(self, model_name='bert-base-uncased', embedding_dim=768, out_dim = 256):
        super().__init__()
        self.encoder = SentenceTransformer(model_name)
        self.encoder_dim = embedding_dim    
        self.out_dim = out_dim
        self.projection = nn.Sequential(
            nn.Linear(self.encoder_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, out_dim)
        )
    def forward(self,texts: List[str]):
        with torch.no_grad():  # Prevents building graph during encoding
            emb = self.encoder.encode(texts, convert_to_tensor=True, normalize_embeddings=False)
        emb = emb.clone().detach().requires_grad_(True)  # Make it autograd-friendly
        emb = self.projection(emb)
        emb = nn.functional.normalize(emb, p=2, dim=-1)
        return emb
    

In [11]:
from torch.nn import TripletMarginLoss

model = TripletEncoder().to('cuda')
criterion = TripletMarginLoss(margin=1.0, p=2)

# Example batch (strings)
anchor_texts = ["some anchor sentence"]
pos_texts    = ["similar sentence"]
neg_texts    = ["dissimilar sentence"]

anchor_emb = model(anchor_texts)
pos_emb = model(pos_texts)
neg_emb = model(neg_texts)

loss = criterion(anchor_emb, pos_emb, neg_emb)
print( loss.item() )
loss.backward()


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


0.8456010222434998


In [12]:
import torch 
from torch.utils.data import Dataset
 
class reddit_dataset(Dataset):
    def __init__(self , df):
        super().__init__()  
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        anchor = f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['body']}"
        text1 = f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['positive_example_1']}"
        text2= f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['negative_example_1']}"
        
        
        if row['rule_violation'] == 1:
            pos = text1
            neg = text2
        else:
            pos = text2
            neg = text1
        
        return anchor, pos, neg
    

In [13]:
from torch.utils.data import DataLoader

dataset = reddit_dataset(df)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [14]:
next(iter(dataloader))

[('GlobalOffensive [SEP] No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. [SEP] The t-shirt can be purchased on http://www.hitpoint.tv/scream-one-tap-t-shirt. Enjoy!',
  'Showerthoughts [SEP] No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. [SEP] tight hole await your big brother check it out at this point http://sh.ors.it/1rJfB',
  "politics [SEP] No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. [SEP] If it's gonna be Hillary and Trump in November we need to have a revolution. Vote Austin Petersen!! #Petersen2016 #AP4LP #Libertarian #DumpTrump #HillaryForPrison #TaxationIsTheft austinpetersen2016.com",
  'AskReddit [SEP] No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. [SEP] Oh that\'s interesting. You should check out my instagram page "dailyetymology" for more cool e

In [19]:
from torch.nn import TripletMarginLoss
import torch.optim as optim

model = TripletEncoder().to('cuda')
criterion = TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
print("stated training...")
for epoch in range(10):  # Example for 1 epoch
    model.train()
    total_loss = 0
    for anchor_texts, pos_texts, neg_texts in dataloader:
        anchor_emb = model(anchor_texts)
        pos_emb = model(pos_texts)
        neg_emb = model(neg_texts)

        loss = criterion(anchor_emb, pos_emb, neg_emb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


stated training...
Epoch 1, Loss: 0.8979239224448917
Epoch 2, Loss: 0.6645168429753912
Epoch 3, Loss: 0.5599706908614617
Epoch 4, Loss: 0.5152522358368701
Epoch 5, Loss: 0.48206209555620283
Epoch 6, Loss: 0.43473090161019423
Epoch 7, Loss: 0.4396047693068587
Epoch 8, Loss: 0.40175561642083596
Epoch 9, Loss: 0.34416784627700414
Epoch 10, Loss: 0.31960777363439247


In [20]:
torch.save(model.state_dict(), "triplet_encoder_weights.pth")

In [15]:

# Count trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(model)

Total parameters: 110,073,088
Trainable parameters: 110,073,088
TripletEncoder(
  (encoder): SentenceTransformer(
    (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (projection): Sequential(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
  )
)


In [16]:
import torch 
from torch.utils.data import Dataset
 
class reddit_dataset2(Dataset):
    def __init__(self , df):
        super().__init__()  
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        anchor = f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['body']}"
        text1  = [f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['positive_example_1']}" ,f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['positive_example_2']}" ]
        text2  = [f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['negative_example_1']}" ,f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['negative_example_2']}" ]
        
        
        if row['rule_violation'] == 1:
            pos = text1
            neg = text2
        else:
            pos = text2
            neg = text1
        
        return anchor, pos, neg
    

In [28]:
from torch.utils.data import DataLoader

dataset = reddit_dataset2(df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [29]:
next(iter(dataloader))

[("relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] Wow this is FUCKED. Sue her for half of all the bills you've paid for, and get the fuck out. This lady is a selfish bitch. I almost feel like she wants to end the relationship and is trying to force you to end it because she's an avoidant asshole. I can't think of any other reason to be this audacious.",),
 [("relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] I would keep it to yourself. If it comes up, pretend its new. Admitting to a fraudulent enlistment (Article 83) would be grounds for a less than honorable discharge. Depending on your CO, the punishment can vary. Try a military sub. I'm sure there will be someone on there that has been in the same situation.",),
  ('relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] > Police told me to pull over and PD would do their job. "Their job will be to arrest me after I shoot this motherfucker whe

In [19]:
from torch.nn import TripletMarginLoss
import torch.optim as optim

train_dataset = reddit_dataset2(df)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

model = TripletEncoder().to('cuda')
criterion = TripletMarginLoss(margin=1.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(3):
    model.train()
    total_loss = 0
    for anchors, positives, negatives in train_loader:
        optimizer.zero_grad()

        # encode anchors
        anchor_emb = model(list(anchors))  # (B, D)
        print(f"anchor_emb shape: {anchor_emb.shape} ")
        # flatten positives/negatives
        flat_pos = [p for plist in positives for p in plist]
        flat_neg = [n for nlist in negatives for n in nlist]
        print(f"flat_pos length: {len(flat_pos)} , flat_neg length: {len(flat_neg)} ")
        
        pos_emb = model(flat_pos)  # (B*2, D)
        neg_emb = model(flat_neg)  # (B*2, D)
        print(f"pos_emb shape: {pos_emb.shape} , neg_emb shape: {neg_emb.shape} ")  
        # reshape back into (B, num_pos, D)
        
        pos_emb = pos_emb.view(len(anchors), -1, pos_emb.size(-1))
        neg_emb = neg_emb.view(len(anchors), -1, neg_emb.size(-1))
        print(f"reshaped pos_emb shape: {pos_emb.shape} , reshaped neg_emb shape: {neg_emb.shape} ")
        # expand anchors to match (B, num_pos, num_neg, D)
        a = anchor_emb.unsqueeze(1).unsqueeze(2)        # (B, 1, 1, D)
        p = pos_emb.unsqueeze(2)                        # (B, num_pos, 1, D)
        n = neg_emb.unsqueeze(1)                        # (B, 1, num_neg, D)
        print(f"expanded a shape: {a.shape} , expanded p shape: {p.shape} , expanded n shape: {n.shape} ")
        # broadcast: (B, num_pos, num_neg, D)
        a = a.expand(-1, p.size(1), n.size(2), -1)
        p = p.expand(-1, p.size(1), n.size(2), -1)
        n = n.expand(-1, p.size(1), n.size(2), -1)
        print(f"broadcasted a shape: {a.shape} , broadcasted p shape: {p.shape} , broadcasted n shape: {n.shape} ")
        # collapse triplets into batch dimension
        a = a.reshape(-1, a.size(-1))
        p = p.reshape(-1, p.size(-1))
        n = n.reshape(-1, n.size(-1))
        print(f"collapsed a shape: {a.shape} , collapsed p shape: {p.shape} , collapsed n shape: {n.shape} ")
        # compute triplet loss in one go
        loss = criterion(a, p, n)
        # loss.backward()
        # optimizer.step()
        
        break
        # total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")
    break


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


anchor_emb shape: torch.Size([1, 256]) 
flat_pos length: 2 , flat_neg length: 2 
pos_emb shape: torch.Size([2, 256]) , neg_emb shape: torch.Size([2, 256]) 
reshaped pos_emb shape: torch.Size([1, 2, 256]) , reshaped neg_emb shape: torch.Size([1, 2, 256]) 
expanded a shape: torch.Size([1, 1, 1, 256]) , expanded p shape: torch.Size([1, 2, 1, 256]) , expanded n shape: torch.Size([1, 1, 2, 256]) 
broadcasted a shape: torch.Size([1, 2, 2, 256]) , broadcasted p shape: torch.Size([1, 2, 2, 256]) , broadcasted n shape: torch.Size([1, 2, 2, 256]) 
collapsed a shape: torch.Size([4, 256]) , collapsed p shape: torch.Size([4, 256]) , collapsed n shape: torch.Size([4, 256]) 
Epoch 1, Loss: 0.0000


In [ ]:
from torch.nn import TripletMarginLoss , TripletMarginWithDistanceLoss
import torch.optim as optim
from tqdm import tqdm 
import torch.nn.functional as F

train_dataset = reddit_dataset2(df)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

model = TripletEncoder().to('cuda')
# criterion = TripletMarginLoss(margin=1.0, p=2)
criterion = TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.3   # smaller margin usually works better for cosine
)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(10):
    model.train()
    total_loss = 0
    for anchors, positives, negatives in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()

        # encode anchors
        anchor_emb = model(list(anchors))  # (B, D)
        
        # flatten positives/negatives
        flat_pos = [p for plist in positives for p in plist]
        flat_neg = [n for nlist in negatives for n in nlist]
        
        pos_emb = model(flat_pos)  # (B*2, D)
        neg_emb = model(flat_neg)  # (B*2, D)
        # reshape back into (B, num_pos, D)
        
        pos_emb = pos_emb.view(len(anchors), -1, pos_emb.size(-1))
        neg_emb = neg_emb.view(len(anchors), -1, neg_emb.size(-1))
        # expand anchors to match (B, num_pos, num_neg, D)
        a = anchor_emb.unsqueeze(1).unsqueeze(2)        # (B, 1, 1, D)
        p = pos_emb.unsqueeze(2)                        # (B, num_pos, 1, D)
        n = neg_emb.unsqueeze(1)                        # (B, 1, num_neg, D)
        
        # broadcast: (B, num_pos, num_neg, D)
        a = a.expand(-1, p.size(1), n.size(2), -1)
        p = p.expand(-1, p.size(1), n.size(2), -1)
        n = n.expand(-1, p.size(1), n.size(2), -1)
        
        # collapse triplets into batch dimension
        a = a.reshape(-1, a.size(-1))
        p = p.reshape(-1, p.size(-1))
        n = n.reshape(-1, n.size(-1))
        
        # compute triplet loss in one go
        loss = criterion(a, p, n)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.
Epoch 1:  61%|██████    | 1233/2029 [08:45<06:38,  2.00it/s]

In [21]:
torch.save(model.state_dict(), "triplet_encoder_weights2.pth")

In [36]:
import torch
from torch.nn.functional import cosine_similarity

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load trained triplet model
model = TripletEncoder().to(device)
model.load_state_dict(torch.load("triplet_encoder_weights2.pth", map_location=device))
model.eval()

def get_embedding(text, model):
    with torch.no_grad():
        emb = model.encoder.encode([text], convert_to_tensor=True, normalize_embeddings=True)
        emb = model.projection(emb)   # use same projection as training
    return emb


def predict(anchor_text, positives, negatives, model):
    anchor_emb = get_embedding(anchor_text, model)

    pairs = []
    # positive pairs
    for i, p in enumerate(positives, 1):
        p_emb = get_embedding(p, model)
        sim = cosine_similarity(anchor_emb, p_emb).item()
        pairs.append((f"positive_{i}", sim, "POSITIVE"))

    # negative pairs
    for i, n in enumerate(negatives, 1):
        n_emb = get_embedding(n, model)
        sim = cosine_similarity(anchor_emb, n_emb).item()
        pairs.append((f"negative_{i}", sim, "NEGATIVE"))

    # choose max similarity
    best = max(pairs, key=lambda x: x[1])

    return {
        "all_pairs": pairs,   # list of (name, score, label)
        "prediction": best[2],
        "best_pair": best
    }


# -------- Example --------
anchor = "relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] Wow this is FUCKED. Sue her for half of all the bills you've paid for, and get the fuck out. This lady is a selfish bitch. I almost feel like she wants to end the relationship and is trying to force you to end it because she's an avoidant asshole. I can't think of any other reason to be this audacious."
 
positives = ["relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] I would keep it to yourself. If it comes up, pretend its new. Admitting to a fraudulent enlistment (Article 83) would be grounds for a less than honorable discharge. Depending on your CO, the punishment can vary. Try a military sub. I'm sure there will be someone on there that has been in the same situation.", "relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] > Police told me to pull over and PD would do their job. Their job will be to arrest me after I shoot this motherfucker when he gets out of his car, so they\'d better find us fast! That reply might have gotten you something akin to the response time you were looking for."]
negatives = ["relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] this is the most infuriating thing I\'ve read all year. my gut tells me to say fuck it all, tie this piece of shit up in the Kentucky wilderness, shoot him in each of the kneecaps with a .22 l, pour gasoline in him and throw a match. my head tells me to make sure his name is always associated as the rapist John and tell anyone he knows what he did, oh and leak his info to all the people online that you can, Facebook too.", "relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] Where is there a site that still works where you can jump the GPS. Is there a FAQ to do this with iPhone or Mac?'"]

out = predict(anchor, positives, negatives, model)

print("All pairs:")
for name, score, label in out["all_pairs"]:
    print(f"{name} → {label}, similarity={score:.4f}")
import torch
from torch.nn.functional import cosine_similarity

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load trained triplet model
model = TripletEncoder().to(device)
model.load_state_dict(torch.load("triplet_encoder_weights2.pth", map_location=device))
model.eval()

def get_embedding(text, model):
    with torch.no_grad():
        emb = model.encoder.encode([text], convert_to_tensor=True, normalize_embeddings=True)
        emb = model.projection(emb)   # use same projection as training
    return emb


def predict(anchor_text, positives, negatives, model):
    anchor_emb = get_embedding(anchor_text, model)

    pairs = []
    # positive pairs
    for i, p in enumerate(positives, 1):
        p_emb = get_embedding(p, model)
        sim = cosine_similarity(anchor_emb, p_emb).item()
        pairs.append((f"positive_{i}", sim, "POSITIVE"))

    # negative pairs
    for i, n in enumerate(negatives, 1):
        n_emb = get_embedding(n, model)
        sim = cosine_similarity(anchor_emb, n_emb).item()
        pairs.append((f"negative_{i}", sim, "NEGATIVE"))

    # choose max similarity
    best = max(pairs, key=lambda x: x[1])

    return {
        "all_pairs": pairs,   # list of (name, score, label)
        "prediction": best[2],
        "best_pair": best
    }


# -------- Example --------
anchor = "relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] Wow this is FUCKED. Sue her for half of all the bills you've paid for, and get the fuck out. This lady is a selfish bitch. I almost feel like she wants to end the relationship and is trying to force you to end it because she's an avoidant asshole. I can't think of any other reason to be this audacious."
 
positives = ["relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] I would keep it to yourself. If it comes up, pretend its new. Admitting to a fraudulent enlistment (Article 83) would be grounds for a less than honorable discharge. Depending on your CO, the punishment can vary. Try a military sub. I'm sure there will be someone on there that has been in the same situation.", "relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] > Police told me to pull over and PD would do their job. Their job will be to arrest me after I shoot this motherfucker when he gets out of his car, so they\'d better find us fast! That reply might have gotten you something akin to the response time you were looking for."]
negatives = ["relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] this is the most infuriating thing I\'ve read all year. my gut tells me to say fuck it all, tie this piece of shit up in the Kentucky wilderness, shoot him in each of the kneecaps with a .22 l, pour gasoline in him and throw a match. my head tells me to make sure his name is always associated as the rapist John and tell anyone he knows what he did, oh and leak his info to all the people online that you can, Facebook too.", "relationships [SEP] No legal advice: Do not offer or request legal advice. [SEP] Where is there a site that still works where you can jump the GPS. Is there a FAQ to do this with iPhone or Mac?'"]

out = predict(anchor, positives, negatives, model)

print("All pairs:")
for name, score, label in out["all_pairs"]:
    print(f"{name} → {label}, similarity={score:.4f}")
guess_score = (
    out["all_pairs"][0][1] +
    out["all_pairs"][1][1] -
    out["all_pairs"][2][1] -
    out["all_pairs"][3][1]
)
print(f"guess score = {guess_score:.4f}")

print(f"\nFinal prediction = {out['prediction']} (best: {out['best_pair']})")
print(f"\nFinal prediction = {out['prediction']} (best: {out['best_pair']})")


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


All pairs:
positive_1 → POSITIVE, similarity=0.9896
positive_2 → POSITIVE, similarity=0.9933
negative_1 → NEGATIVE, similarity=0.9917
negative_2 → NEGATIVE, similarity=0.9813


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


All pairs:
positive_1 → POSITIVE, similarity=0.9896
positive_2 → POSITIVE, similarity=0.9933
negative_1 → NEGATIVE, similarity=0.9917
negative_2 → NEGATIVE, similarity=0.9813
guess score = 0.0098

Final prediction = POSITIVE (best: ('positive_2', 0.9932569265365601, 'POSITIVE'))

Final prediction = POSITIVE (best: ('positive_2', 0.9932569265365601, 'POSITIVE'))


In [50]:
import torch
import pandas as pd
from torch.nn.functional import cosine_similarity
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model
model = TripletEncoder().to(device)
model.load_state_dict(torch.load("triplet_encoder_weights2.pth", map_location=device))
model.eval()


def get_embedding(text, model):
    with torch.no_grad():
        emb = model.encoder.encode([text], convert_to_tensor=True, normalize_embeddings=True)
        emb = model.projection(emb)
    return emb


def predict_row(row, model):
    # Anchor is rule-aware
    anchor = f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['body']}"
    positives = [
        f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['positive_example_1']}",
        f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['positive_example_2']}"
    ]
    negatives = [
        f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['negative_example_1']}",
        f"{row['subreddit']} [SEP] {row['rule']} [SEP] {row['negative_example_2']}"
    ]

    anchor_emb = get_embedding(anchor, model)

    pairs = []
    for i, p in enumerate(positives, 1):
        p_emb = get_embedding(p, model)
        sim = cosine_similarity(anchor_emb, p_emb).item()
        pairs.append((f"positive_{i}", sim, "POSITIVE"))

    for i, n in enumerate(negatives, 1):
        n_emb = get_embedding(n, model)
        sim = cosine_similarity(anchor_emb, n_emb).item()
        pairs.append((f"negative_{i}", sim, "NEGATIVE"))

    best = max(pairs, key=lambda x: x[1])  # pick highest similarity
    diff = sum(score for name, score, label in pairs if label == "POSITIVE") - \
           sum(score for name, score, label in pairs if label == "NEGATIVE")
    return {
        "prediction": best[2],
        "best_pair": best[0],
        "best_score": best[1],
        "all_scores": {name: score for name, score, _ in pairs},
        "diff": diff
    }


def run_inference(df, model, save_path="predictions.csv"):
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        out = predict_row(row, model)
        results.append({
            "row_id": row["row_id"],
            "prediction": out["prediction"],
            "best_pair": out["best_pair"],
            "best_score": out["best_score"],
            "actual":row["rule_violation"],
            "diff": out["diff"],
            **out["all_scores"]  # expand all similarity scores
        })
    results_df = pd.DataFrame(results)
    results_df.to_csv(save_path, index=False)
    return results_df


# Example:
test_df = pd.read_csv("train.csv")
test_df = clean_spaces(test_df)
test_df = test_df.sample(frac=0.1).reset_index(drop=True)  # shuffle
predictions = run_inference(test_df,model, save_path="test_predictions.csv")
print(predictions.head())


No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.
100%|██████████| 203/203 [04:10<00:00,  1.24s/it]

   row_id prediction   best_pair  best_score  actual      diff  positive_1  \
0    1067   POSITIVE  positive_2    0.997570       1  0.043545    0.994076   
1    1300   POSITIVE  positive_2    0.995403       1  0.013228    0.994625   
2     991   NEGATIVE  negative_1    0.992189       0 -0.023843    0.977838   
3    2027   POSITIVE  positive_1    0.994966       1  0.021704    0.994966   
4     339   POSITIVE  positive_2    0.999541       1  0.046031    0.996249   

   positive_2  negative_1  negative_2  
0    0.997570    0.967992    0.980109  
1    0.995403    0.987678    0.989121  
2    0.974779    0.992189    0.984272  
3    0.966139    0.963979    0.975421  
4    0.999541    0.970277    0.979483  


In [51]:
predictions

,row_id,prediction,best_pair,best_score,actual,diff,positive_1,positive_2,negative_1,negative_2
0,1067,POSITIVE,positive_2,0.997570,1,0.043545,0.994076,0.997570,0.967992,0.980109
1,1300,POSITIVE,positive_2,0.995403,1,0.013228,0.994625,0.995403,0.987678,0.989121
2,991,NEGATIVE,negative_1,0.992189,0,-0.023843,0.977838,0.974779,0.992189,0.984272
3,2027,POSITIVE,positive_1,0.994966,1,0.021704,0.994966,0.966139,0.963979,0.975421
4,339,POSITIVE,positive_2,0.999541,1,0.046031,0.996249,0.999541,0.970277,0.979483
...,...,...,...,...,...,...,...,...,...,...
198,2026,NEGATIVE,negative_2,0.997234,1,-0.009271,0.992639,0.991406,0.996082,0.997234
199,1005,POSITIVE,positive_2,0.997052,1,-0.002558,0.991180,0.997052,0.994541,0.996249
200,1402,NEGATIVE,negative_2,0.992910,0,-0.024139,0.966217,0.991537,0.988982,0.992910
201,1244,NEGATIVE,negative_1,0.997621,0,-0.005252,0.991551,0.993223,0.997621,0.992405


In [57]:
predictions["vote"] = predictions['prediction'].apply(lambda x: 1 if x == "POSITIVE" else 0)
len(predictions[predictions["actual"] == predictions["vote"]])/len(predictions)

0.8472906403940886

In [58]:
predictions["vote"] = predictions['diff'].apply(lambda x: 1 if x > 0 else 0)
len(predictions[predictions["actual"] == predictions["vote"]])/len(predictions)

0.8078817733990148